# Component: Model I/O

In [2]:
# !pip install langchain langchain-openai

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [1]:
import os
# os.getenv("OPENAI_API_KEY")
# os.getenv("HF_TOKEN")

### PromptTemplate

In [5]:
# prompttemplate

from langchain import PromptTemplate

template="{product}를 홍보하기 위한 재미있고, 신선한 광고문구를 작성해 주세요."   # product가 동적변수. f-string과는 다름. 동적변수로 만들려면 중괄호를 씌워주면 됨.

prompt = PromptTemplate(
    template=template,
    input_variables=['product'] # 동적변수(변경이 될 동적인 요소.) / 입력변수를 동적변수로 받아올 수 있게끔
)

print(prompt.format(product='카메라'))

카메라를 홍보하기 위한 재미있고, 신선한 광고문구를 작성해 주세요.


In [6]:
# FewShotPromptTemplate
from langchain.prompts import FewShotPromptTemplate

# 예시 데이터 
examples = [
    {"question": "2 + 2는 무엇인가요?", "answer": "2 + 2 = 4"}, 
    {"question": "3 + 4는 무엇인가요?", "answer": "3 + 4 = 7"}
]

# 예시를 어떻게 출력할지 포맷 지정
example_prompt = PromptTemplate(
    template="Q: {question}\nA: {answer}",
    input_variables=['question', 'answer']
)

# FewShotPromptTemplate 생성
fewshot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="다음 계산 문제를 해결하세요.",          # 앞에 붙는 설명
    suffix="Q: {question}은 무엇인가요?\nA:",       # 실제 사용자 입력
    input_variables=['question']
) 

# 사용자 질문으로 프롬프트 생성 
print(fewshot_prompt.format(question='34 + 78'))

다음 계산 문제를 해결하세요.

Q: 2 + 2는 무엇인가요?
A: 2 + 2 = 4

Q: 3 + 4는 무엇인가요?
A: 3 + 4 = 7

Q: 34 + 78은 무엇인가요?
A:


In [7]:
# ChatPromptTemplate - System, Human, AI 유형별 메세지 작성
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

sys_msg = SystemMessagePromptTemplate.from_template('당신은 친절한 챗봇입니다.')
hm_msg = HumanMessagePromptTemplate.from_template('질문: {question}')
msg = ChatPromptTemplate.from_messages([sys_msg, hm_msg])   # 여러개의 메세지 템플릿을 조합
print(msg)

msg.format_messages(question='AI를 배우려면 무엇부터 해야 하나요?')


input_variables=['question'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 친절한 챗봇입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='질문: {question}'), additional_kwargs={})]


[SystemMessage(content='당신은 친절한 챗봇입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='질문: AI를 배우려면 무엇부터 해야 하나요?', additional_kwargs={}, response_metadata={})]

### OutputParser

In [8]:
# OutputParser 인스턴트 및 PromptTemplate 인스턴스 생성
from langchain_openai import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions() # 파싱해주는 형식 정보에 대한 내용을 가지고 온다.
# 프롬프트를 날려줄때부터 사용 가능 -> PromptTemplate 안에 구성해야한다

prompt_tpl = PromptTemplate(
    template="{subject} 5개의 팀을 알려주세요.\n형식 지정: {format}",   # 파싱을 하려면 형식이 맞아야하기 때문에 그 형식을 맞춰주기 위함
    input_variables=['subject'],                        # input_variables : 사용자 입력 변수
    partial_variables={'format': format_instructions}   # partial_variables : 고정 설정 변수
)

In [9]:
# 프롬프트 생성
query = "한국의 프로야구팀"
prompt = prompt_tpl.format(subject=query)
print(prompt)   # format_instruction

한국의 프로야구팀 5개의 팀을 알려주세요.
형식 지정: Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [10]:
# open ai api를 사용하기 위한 인스턴스 생성 및 요청 처리
model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    max_tokens=2048
)

response = model.invoke(prompt) # 질의를 보내는 메서드 / prompt의 질의를 보냄

In [11]:
print(response.content)         # response의 content 안에 우리가 원하는 답변이 있어서 꺼내옴
print(type(response.content))   # response의 type은 기본적으로 str이다.

두산 베어스, LG 트윈스, 삼성 라이온즈, 키움 히어로즈, NC 다이노스
<class 'str'>


In [12]:
print(output_parser.parse(response.content))        # 문자열인 상태를 parsing하여 리스트의 형태로 변환
print(type(output_parser.parse(response.content)))  # 리스트
print(type(response.content))                       # 문자열

['두산 베어스', 'LG 트윈스', '삼성 라이온즈', '키움 히어로즈', 'NC 다이노스']
<class 'list'>
<class 'str'>


### HuggingFace Model

In [13]:
# !pip install langchain_huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

end_point = HuggingFaceEndpoint(
    repo_id="Bllossom/llama-3.2-Korean-Bllossom-3B",    # 허깅페이스에 저장된 모델의 repositoy
    task='text-generation',                             # 수행하고자하는 작업의 유형
    max_new_tokens=1024,                                # 생성할 최대 토큰 수
    huggingfacehub_api_token=''
)

hf_model = ChatHuggingFace(
    llm=end_point,
    verbose=True
)

hf_model.invoke('저는 아침으로 사과를 먹었습니다. 저는 아침에 뭘 먹었을까요?')

c:\Users\USER\anaconda3\envs\vectordb_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


AIMessage(content='आ침에 사과를 먹었습니다.', additional_kwargs={}, response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=11, prompt_tokens=98, total_tokens=109), 'model': '', 'finish_reason': 'stop'}, id='run-c11cbc19-2318-4085-af2c-04e35abd65f0-0')

In [2]:
# from langchain.model_laboratory import ModelLaboratory

# model_lab = ModelLaboratory.from_llms([model, hf_model])
# model_lab.compare('아침에 사과를 먹는 것의 효과를 알려주세요.')